In [8]:
import time
import random
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# =========================
# CONFIG
# =========================
TOKEN_URL = "https://login.impect.com/auth/realms/production/protocol/openid-connect/token"
BASE_API_URL = "https://api.impect.com"

USERNAME = "marclambertsmedia@gmail.com"
PASSWORD = "Meneertosti@1!"  # rotate leaked password ASAP

OUTPUT_CSV = "/Users/user/IMPECT/player_kpis_iteration_1421.csv"

ITERATION_ID = 1421
KPI_LANGUAGE = "en"

TIMEOUT_SECONDS = 30

# Keep low to avoid 429s
MAX_WORKERS = 3
BASE_SLEEP_SECONDS = 0.2

MAX_RETRIES = 8
BACKOFF_BASE = 0.75
BACKOFF_CAP = 30.0
# =========================


def unwrap_data(obj):
    return obj["data"] if isinstance(obj, dict) and "data" in obj else obj


def get_access_token(session: requests.Session, username: str, password: str) -> str:
    payload = {
        "client_id": "api",
        "grant_type": "password",
        "username": username,
        "password": password,
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    r = session.post(TOKEN_URL, data=payload, headers=headers, timeout=TIMEOUT_SECONDS)
    r.raise_for_status()
    return r.json()["access_token"]


def get_with_retry(session: requests.Session, url: str, token: str):
    headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}

    for attempt in range(MAX_RETRIES + 1):
        r = session.get(url, headers=headers, timeout=TIMEOUT_SECONDS)

        if r.status_code < 400:
            return r.json()

        if r.status_code in (429, 500, 502, 503, 504):
            retry_after = r.headers.get("Retry-After")
            wait = None

            if retry_after:
                try:
                    wait = float(retry_after)
                except ValueError:
                    wait = None

            if wait is None:
                wait = min(BACKOFF_CAP, BACKOFF_BASE * (2 ** attempt))
                wait += random.uniform(0, 0.35 * wait)

            if attempt >= MAX_RETRIES:
                r.raise_for_status()

            time.sleep(wait)
            continue

        r.raise_for_status()

    raise RuntimeError("Retry loop exited unexpectedly")


def fetch_kpi_defs(session: requests.Session, token: str, language: str):
    url = f"{BASE_API_URL}/v5/customerapi/kpis?language={language}"
    return unwrap_data(get_with_retry(session, url, token))


def fetch_iteration_squads(session: requests.Session, iteration_id: int, token: str):
    url = f"{BASE_API_URL}/v5/customerapi/iterations/{iteration_id}/squads"
    return unwrap_data(get_with_retry(session, url, token))


def fetch_iteration_players(session: requests.Session, iteration_id: int, token: str):
    url = f"{BASE_API_URL}/v5/customerapi/iterations/{iteration_id}/players"
    return unwrap_data(get_with_retry(session, url, token))


def fetch_player_kpis_for_squad(session: requests.Session, iteration_id: int, squad_id: int, token: str):
    url = f"{BASE_API_URL}/v5/customerapi/iterations/{iteration_id}/squads/{squad_id}/player-kpis"
    return unwrap_data(get_with_retry(session, url, token))


def extract_long_rows(iteration_id: int, squad_id: int, squad_name: str, items: list):
    out = []
    for item in items:
        matches = item.get("matches")
        if matches is None or (isinstance(matches, str) and matches.strip() == ""):
            matches = 0

        # playerId is what we need for lookup
        player_obj = item.get("player") if isinstance(item, dict) else None
        p = player_obj if isinstance(player_obj, dict) else (item if isinstance(item, dict) else {})
        player_id = p.get("id") or item.get("playerId")

        for kv in (item.get("kpis") or []):
            out.append({
                "iterationId": iteration_id,
                "squadId": squad_id,
                "squadName": squad_name,
                "playerId": player_id,
                "matches": matches,
                "kpiId": kv.get("kpiId"),
                "value_raw": kv.get("value"),  # e.g. "27,513527"
            })
    return out


def main():
    session = requests.Session()
    t0 = time.time()

    token = get_access_token(session, USERNAME, PASSWORD)
    print("✅ Access token retrieved")

    # --- KPI definitions (for column names after pivot) ---
    kpi_defs = fetch_kpi_defs(session, token, KPI_LANGUAGE)
    kpi_df = pd.json_normalize(kpi_defs if isinstance(kpi_defs, list) else [])
    label_col = "details.label" if "details.label" in kpi_df.columns else "name"

    kpi_lookup = (
        kpi_df.rename(columns={"id": "kpiId", label_col: "kpiLabel", "name": "kpiTechnicalName"})[
            ["kpiId", "kpiLabel", "kpiTechnicalName"]
        ]
        .drop_duplicates()
    )
    kpi_lookup["kpiId"] = pd.to_numeric(kpi_lookup["kpiId"], errors="coerce")

    rename_map = dict(
        zip(
            kpi_lookup["kpiId"],
            (
                kpi_lookup["kpiLabel"]
                .fillna(kpi_lookup["kpiTechnicalName"])
                .fillna("kpi")
                + " (" + kpi_lookup["kpiId"].astype("Int64").astype(str) + ")"
            ),
        )
    )

    # --- Player master data (THIS is what you were missing) ---
    players = fetch_iteration_players(session, ITERATION_ID, token)
    players_df = pd.json_normalize(players if isinstance(players, list) else [])
    # Ensure expected columns exist even if API omits some
    expected_player_cols = [
        "id", "firstname", "lastname", "commonname", "birthdate", "birthplace",
        "leg", "countryIds", "gender", "idMappings"
    ]
    for c in expected_player_cols:
        if c not in players_df.columns:
            players_df[c] = None

    players_df = players_df[expected_player_cols].rename(columns={"id": "playerId"})
    players_df["playerId"] = pd.to_numeric(players_df["playerId"], errors="coerce")

    print(f"✅ Players loaded from /players: {len(players_df)}")

    # --- Squads ---
    squads = fetch_iteration_squads(session, ITERATION_ID, token)
    squads_lookup = [{"squadId": s.get("id"), "squadName": s.get("name")} for s in (squads or [])]
    print(f"✅ Squads loaded: {len(squads_lookup)}")

    # --- Fetch player-kpis (per squad) ---
    long_rows = []

    def worker(squad):
        sid = squad["squadId"]
        sname = squad["squadName"]
        if sid is None:
            return []
        if BASE_SLEEP_SECONDS:
            time.sleep(BASE_SLEEP_SECONDS + random.uniform(0, BASE_SLEEP_SECONDS))
        items = fetch_player_kpis_for_squad(session, ITERATION_ID, int(sid), token)
        if not isinstance(items, list):
            return []
        return extract_long_rows(ITERATION_ID, sid, sname, items)

    t_fetch_start = time.time()
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        futures = [ex.submit(worker, s) for s in squads_lookup]
        for fut in as_completed(futures):
            long_rows.extend(fut.result())
    print(f"⏱️ Fetch seconds: {time.time() - t_fetch_start:.1f}")

    player_long = pd.DataFrame(long_rows)
    print(f"✅ Player KPI rows: {len(player_long)}")
    if player_long.empty:
        raise RuntimeError("No player KPI rows returned.")

    # --- Conversions (decimal comma ALWAYS) ---
    player_long["playerId"] = pd.to_numeric(player_long["playerId"], errors="coerce")
    player_long["kpiId"] = pd.to_numeric(player_long["kpiId"], errors="coerce")
    player_long["matches"] = pd.to_numeric(player_long["matches"], errors="coerce").fillna(0)

    s = (
        player_long["value_raw"]
        .astype("string")
        .str.replace(" ", "", regex=False)
        .str.replace(",", ".", regex=False)
    )
    player_long["value"] = pd.to_numeric(s, errors="coerce")
    print("🔎 value null rate:", float(player_long["value"].isna().mean()))

    player_long = player_long.dropna(subset=["playerId", "kpiId"])
    player_long = player_long.drop_duplicates(
        subset=["iterationId", "squadId", "playerId", "matches", "kpiId"],
        keep="first"
    )

    # --- Pivot wide ---
    wide = (
        player_long.pivot(
            index=["iterationId", "squadId", "squadName", "playerId", "matches"],
            columns="kpiId",
            values="value"
        )
        .reset_index()
    )
    wide.columns.name = None
    print(f"✅ Wide table: {wide.shape[0]} rows × {wide.shape[1]} cols")

    # Rename KPI columns (cheap, after pivot)
    wide = wide.rename(columns={k: v for k, v in rename_map.items() if k in wide.columns})

    # --- Join player identity ---
    wide = wide.merge(players_df, how="left", on="playerId")

    # Put identity columns up front (nice for CSV)
    front_cols = ["iterationId", "squadId", "squadName", "playerId", "commonname", "firstname", "lastname", "matches",
                  "birthdate", "birthplace", "leg", "countryIds", "gender", "idMappings"]
    existing_front = [c for c in front_cols if c in wide.columns]
    other_cols = [c for c in wide.columns if c not in existing_front]
    wide = wide[existing_front + other_cols]

    # --- Write CSV (filled) ---
    wide.to_csv(
        OUTPUT_CSV,
        index=False,
        encoding="utf-8",
        float_format="%.6f",
    )

    print("\n✅ CSV SAVED")
    print(f"📁 {OUTPUT_CSV}")
    print(f"⏱️ Total seconds: {time.time() - t0:.1f}")

    # Quick check: how many rows found a player match?
    match_rate = 1.0 - wide["commonname"].isna().mean() if "commonname" in wide.columns else None
    if match_rate is not None:
        print(f"🔎 player identity match rate: {match_rate:.3f}")


if __name__ == "__main__":
    main()


✅ Access token retrieved
✅ Players loaded from /players: 504
✅ Squads loaded: 16
⏱️ Fetch seconds: 14.7
✅ Player KPI rows: 661102
🔎 value null rate: 0.0
✅ Wide table: 431 rows × 1420 cols

✅ CSV SAVED
📁 /Users/user/IMPECT/player_kpis_iteration_1421.csv
⏱️ Total seconds: 21.3
🔎 player identity match rate: 1.000


In [7]:
"""
IMPECT — Build radars from *your CSV* (iteration 1421)
=====================================================
CSV-only (no API):
  ✅ Loads your CSV robustly
  ✅ Parses IMPECT number formatting (e.g. 65.506.424 -> 65.506424)
  ✅ Computes derived metrics safely (NO ZeroDivisionError)
  ✅ Auto-builds position profiles using ONLY columns that exist in your CSV
  ✅ Generates radar+table PNGs
  ✅ META_COLS guard — name columns are never clobbered by impect_float
  ✅ 10-12 metrics per profile, every KPI ID verified against this CSV

Run:
  python impect_player_from_csv.py
"""

import os
import csv
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ─────────────────────────────────────────────
# CONFIG
# ─────────────────────────────────────────────
CSV_PATH = "/Users/user/IMPECT/player_kpis_iteration_1421.csv"
OUT_DIR  = "/Users/user/IMPECT/impect_outputs"

TARGET_PLAYER  = "Lukas Letenay"
TARGET_PROFILE = None          # e.g. "Winger" or None = auto

BENCHMARK_BY_POSITION = True
MIN_POSITION_SAMPLE   = 10
MIN_METRICS_PER_RADAR = 6
MAX_METRICS_PER_RADAR = 12

# ─────────────────────────────────────────────
# COLOUR PALETTE  (dark theme)
# ─────────────────────────────────────────────
BG            = "#1A1A1A"
RADAR_BG      = "#222222"
GRID_COLOR    = "#333333"
PLAYER_COL    = "#F5C518"
LEAGUE_COL    = "#5A5A5A"
TEXT_LIGHT    = "#E0E0E0"
TEXT_DIM      = "#888888"
TABLE_ROW_ALT = "#2A2A2A"
TABLE_BORDER  = "#3A3A3A"
PCT_HIGH      = "#4CAF50"
PCT_MID       = "#F5C518"
PCT_LOW       = "#E53935"
ACCENT_LINE   = "#F5C518"

# ─────────────────────────────────────────────
# META_COLS — never passed through impect_float
# ─────────────────────────────────────────────
META_COLS = {
    "commonname", "firstname", "lastname", "squadName", "positions",
    "playerId", "squadId", "birthdate", "birthplace", "leg",
    "countryIds", "gender", "idMappings", "iterationId",
}

# ─────────────────────────────────────────────
# DERIVED METRICS
# ─────────────────────────────────────────────
DERIVED = {
    "Aerial Win %": {
        "num":   ["Won Aerial Duels (96)"],
        "denom": ["Won Aerial Duels (96)", "Lost Aerial Duels (97)"],
        "scale": 100,
    },
    "Ground Duel Win %": {
        "num":   ["Won Ground Duels (94)"],
        "denom": ["Won Ground Duels (94)", "Lost Ground Duels (95)"],
        "scale": 100,
    },
    "Pass Accuracy %": {
        "num":   ["Successful Passes (90)"],
        "denom": ["Successful Passes (90)", "Unsuccessful Passes (91)"],
        "scale": 100,
    },
    "Shots on Target %": {
        "num":   ["Total Shots (100)"],
        "denom": ["Total Shots (100)"],
        "scale": 100,
    },
    # sum of low + high cross
    "Crosses": {
        "num":   [
            "SUCCESSFUL_PASSES_BY_ACTION_LOW_CROSS (293)",
            "SUCCESSFUL_PASSES_BY_ACTION_HIGH_CROSS (294)",
        ],
        "denom": None,
        "scale": 1,
    },
    # sum of low + high cross bypasses (opponents)
    "Cross Bypasses": {
        "num":   [
            "BYPASSED_OPPONENTS_BY_ACTION_LOW_CROSS (110)",
            "BYPASSED_OPPONENTS_BY_ACTION_HIGH_CROSS (111)",
        ],
        "denom": None,
        "scale": 1,
    },
    # sum of low + diagonal pass bypasses
    "Pass Bypasses": {
        "num":   [
            "BYPASSED_OPPONENTS_BY_ACTION_LOW_PASS (106)",
            "BYPASSED_OPPONENTS_BY_ACTION_DIAGONAL_PASS (107)",
        ],
        "denom": None,
        "scale": 1,
    },
    # sum of low + diagonal pass defender bypasses
    "Pass Def Bypasses": {
        "num":   [
            "BYPASSED_DEFENDERS_BY_ACTION_LOW_PASS (167)",
            "BYPASSED_DEFENDERS_BY_ACTION_DIAGONAL_PASS (168)",
        ],
        "denom": None,
        "scale": 1,
    },
    # touches in final third + opponent box
    "Advanced Touches": {
        "num":   [
            "OFFENSIVE_TOUCHES_IN_PITCH_POSITION_FINAL_THIRD (598)",
            "OFFENSIVE_TOUCHES_IN_PITCH_POSITION_OPPONENT_BOX (599)",
        ],
        "denom": None,
        "scale": 1,
    },
    # defensive touches via interception + duel
    "Defensive Actions": {
        "num":   [
            "DEFENSIVE_TOUCHES_BY_ACTION_INTERCEPTION (611)",
            "DEFENSIVE_TOUCHES_BY_ACTION_DUEL (612)",
        ],
        "denom": None,
        "scale": 1,
    },
    # bypasses in final third + opponent box (opponents)
    "Final Third Bypasses": {
        "num":   [
            "BYPASSED_OPPONENTS_FROM_PITCH_POSITION_FINAL_THIRD (145)",
            "BYPASSED_OPPONENTS_FROM_PITCH_POSITION_OPPONENT_BOX (146)",
        ],
        "denom": None,
        "scale": 1,
    },
    # bypasses in final third + opp box (defenders)
    "Final Third Def Bypasses": {
        "num":   [
            "BYPASSED_DEFENDERS_FROM_PITCH_POSITION_FINAL_THIRD (206)",
            "BYPASSED_DEFENDERS_FROM_PITCH_POSITION_OPPONENT_BOX (207)",
        ],
        "denom": None,
        "scale": 1,
    },
    # successful passes from final third + opp box
    "Deep Passes": {
        "num":   [
            "SUCCESSFUL_PASSES_FROM_PITCH_POSITION_FINAL_THIRD (327)",
            "SUCCESSFUL_PASSES_FROM_PITCH_POSITION_OPPONENT_BOX (328)",
        ],
        "denom": None,
        "scale": 1,
    },
    # ball loss from final third + opp box (damage metric)
    "Deep Ball Losses": {
        "num":   [
            "BALL_LOSS_REMOVED_TEAMMATES_FROM_PITCH_POSITION_FINAL_THIRD (682)",
            "BALL_LOSS_REMOVED_TEAMMATES_FROM_PITCH_POSITION_OPPONENT_BOX (683)",
        ],
        "denom": None,
        "scale": 1,
    },
}

# ─────────────────────────────────────────────
# Robust CSV loader
# ─────────────────────────────────────────────
def load_csv(path: str) -> pd.DataFrame:
    try:
        return pd.read_csv(path, sep=";", engine="python", encoding="utf-8")
    except Exception:
        try:
            return pd.read_csv(path, sep=",", engine="python", encoding="utf-8")
        except Exception:
            with open(path, "r", encoding="utf-8", errors="replace") as f:
                sample = f.read(65536)
            dialect = csv.Sniffer().sniff(sample, delimiters=[",", ";", "\t", "|"])
            return pd.read_csv(path, sep=dialect.delimiter, engine="python",
                               encoding="utf-8", on_bad_lines="skip")

# ─────────────────────────────────────────────
# IMPECT number parsing
# ─────────────────────────────────────────────
def impect_float(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return np.nan
    if isinstance(x, (int, float, np.number)):
        return float(x)
    s = str(x).strip()
    if s == "" or s.lower() in {"none", "nan"}:
        return np.nan
    s = s.replace(" ", "")
    if "," in s and "." not in s:
        s = s.replace(",", ".")
    if s.count(".") >= 1:
        parts = s.split(".")
        if len(parts) >= 2 and all(p.isdigit() for p in parts if p != ""):
            int_part = parts[0] if parts[0] else "0"
            dec_part = "".join(parts[1:])
            try:
                return float(f"{int_part}.{dec_part}")
            except ValueError:
                pass
    try:
        return float(s)
    except ValueError:
        return np.nan


def prep_numeric(df: pd.DataFrame, cols) -> pd.DataFrame:
    out = df.copy()
    for c in cols:
        if c is None or c in META_COLS:   # ← skip metadata
            continue
        if c not in out.columns:
            out.loc[:, c] = np.nan
        out.loc[:, c] = out[c].apply(impect_float)
    return out

# ─────────────────────────────────────────────
# Safe division
# ─────────────────────────────────────────────
def safe_divide(num, denom, scale=1.0):
    num_f = pd.to_numeric(num, errors="coerce").astype(float)
    den_f = pd.to_numeric(denom, errors="coerce").astype(float)
    out   = np.full(len(num_f), np.nan, dtype=float)
    np.divide(num_f.to_numpy(), den_f.to_numpy(), out=out, where=(den_f.to_numpy() > 0))
    return out * float(scale)

# ─────────────────────────────────────────────
# Derived metrics
# ─────────────────────────────────────────────
def compute_derived(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    needed = []
    for spec in DERIVED.values():
        needed += spec["num"]
        if spec["denom"] is not None:
            needed += spec["denom"]
    out = prep_numeric(out, needed)

    for name, spec in DERIVED.items():
        if any(c not in out.columns for c in spec["num"]):
            out.loc[:, name] = np.nan
            continue
        num = out[spec["num"]].sum(axis=1)
        if spec["denom"] is None:
            out.loc[:, name] = pd.to_numeric(num, errors="coerce").astype(float) * float(spec["scale"])
            continue
        if any(c not in out.columns for c in spec["denom"]):
            out.loc[:, name] = np.nan
            continue
        denom = out[spec["denom"]].sum(axis=1)
        out.loc[:, name] = safe_divide(num, denom, scale=spec["scale"])
    return out

# ─────────────────────────────────────────────
# Player search
# ─────────────────────────────────────────────
def _norm_name(s: str) -> str:
    if s is None or (isinstance(s, float) and np.isnan(s)):
        return ""
    s = str(s).replace("\u00a0", " ")
    return re.sub(r"\s+", " ", s).strip().lower()

def find_player_rows(df: pd.DataFrame, query: str) -> pd.DataFrame:
    q = _norm_name(query)
    if not q:
        return df.iloc[0:0]
    for c in ["commonname", "firstname", "lastname"]:
        if c not in df.columns:
            df[c] = ""
    cn   = df["commonname"].map(_norm_name)
    fn   = df["firstname"].map(_norm_name)
    ln   = df["lastname"].map(_norm_name)
    full = (fn + " " + ln).str.strip()

    m1 = cn == q
    if m1.any(): return df[m1]
    m2 = full == q
    if m2.any(): return df[m2]
    m3 = (cn.str.contains(q, na=False) | fn.str.contains(q, na=False)
          | ln.str.contains(q, na=False) | full.str.contains(q, na=False))
    return df[m3]

# ─────────────────────────────────────────────
# Position helpers
# ─────────────────────────────────────────────
def positions_column(df):
    if "positions" in df.columns: return "positions"
    if "position" in df.columns:  return "position"
    return None

def normalize_positions_str(x) -> str:
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ""
    return str(x).strip()

def primary_position(pos_str: str) -> str:
    s = normalize_positions_str(pos_str)
    if not s: return ""
    parts = [p.strip() for p in s.replace(";", ",").split(",") if p.strip()]
    return parts[0] if parts else s

def filter_for_profile(df, pos_col, profile_name):
    """Return only rows whose positions string contains at least one of the
    position tokens that map TO this profile.  Falls back to the full df when
    the resulting pool is too small."""
    if not pos_col or profile_name not in PROFILE_TO_POSITIONS:
        return df
    tokens = PROFILE_TO_POSITIONS[profile_name]          # e.g. ["LEFT_WINGER","RIGHT_WINGER"]
    mask = pd.Series(False, index=df.index)
    for t in tokens:
        mask |= df[pos_col].astype(str).str.contains(t, case=False, na=False)
    out = df[mask]
    return out if len(out) >= MIN_POSITION_SAMPLE else df

# ─────────────────────────────────────────────
# KPI column resolver
# ─────────────────────────────────────────────
def kpi_id_from_col(colname: str):
    if not isinstance(colname, str): return None
    m = re.search(r"\((\d+)\)\s*$", colname.strip())
    return int(m.group(1)) if m else None

def resolve_by_kpi_id(kpi_id: int, columns):
    suffix = f"({kpi_id})"
    hits = [c for c in columns if isinstance(c, str) and c.strip().endswith(suffix)]
    return sorted(hits, key=len)[0] if hits else None

def resolve_by_keywords(keywords, columns):
    kws  = [k.lower() for k in keywords if k]
    best = None
    for c in columns:
        if not isinstance(c, str): continue
        cl = c.lower()
        if all(k in cl for k in kws):
            if best is None or len(c) < len(best):
                best = c
    return best

def resolve_any(candidates, columns):
    for cand in candidates:
        if isinstance(cand, int):
            col = resolve_by_kpi_id(cand, columns)
            if col: return col
        elif isinstance(cand, str):
            if cand in columns: return cand
            cid = kpi_id_from_col(cand)
            if cid is not None:
                col = resolve_by_kpi_id(cid, columns)
                if col: return col
            name_part = cand.split("(")[0].strip()
            if name_part:
                col = resolve_by_keywords([name_part], columns)
                if col: return col
        else:
            col = resolve_by_keywords(list(cand), columns)
            if col: return col
    return None

# ─────────────────────────────────────────────
# PROFILE TEMPLATES  —  10-12 metrics each
#
# Every KPI ID here was verified present in iteration-1421 CSV.
# Derived metric names (e.g. "Crosses") must match keys in DERIVED above.
# Format per row:  (display_label, [resolver_candidates…], invert_flag)
# ─────────────────────────────────────────────
PROFILE_TEMPLATES = {
    # ── STRIKER (12 metrics) ──────────────────────────────────────────
    "Striker": [
        ("Goals",                [28],                                          False),
        ("Assists",              [77],                                          False),
        ("xG",                   [82],                                          False),
        ("Non-Shot xG",          [83],                                          False),
        ("Total Shots",          [100],                                         False),
        ("Shots Off Target",     [101],                                         True),   # lower is better
        ("Touches in Box",       [599],                                         False),
        ("Advanced Touches",     ["Advanced Touches"],                          False),
        ("Bypassed Opponents",   [0],                                           False),
        ("Aerial Win %",         ["Aerial Win %"],                              False),
        ("Critical Ball Losses", [49],                                          True),   # lower is better
        ("Deep Ball Losses",     ["Deep Ball Losses"],                          True),   # lower is better
    ],

    # ── WINGER (12 metrics) ───────────────────────────────────────────
    "Winger": [
        ("Goals",                [28],                                          False),
        ("Assists",              [77],                                          False),
        ("xG",                   [82],                                          False),
        ("Non-Shot xG",          [83],                                          False),
        ("Crosses",              ["Crosses"],                                   False),
        ("Cross Bypasses",       ["Cross Bypasses"],                            False),
        ("Dribble Bypasses",     [87],                                          False),
        ("Defenders Beaten",     [88],                                          False),
        ("Touches in Box",       [599],                                         False),
        ("Final Third Bypasses", ["Final Third Bypasses"],                      False),
        ("Ball Losses",          [22],                                          True),
        ("Critical Ball Losses", [49],                                          True),
    ],

    # ── ATTACKING MID (12 metrics) ────────────────────────────────────
    "Attacking Mid": [
        ("Assists",              [77],                                          False),
        ("xG",                   [82],                                          False),
        ("Non-Shot xG",          [83],                                          False),
        ("Pass Bypasses",        ["Pass Bypasses"],                             False),
        ("Pass Def Bypasses",    ["Pass Def Bypasses"],                         False),
        ("Bypassed Opponents",   [0],                                           False),
        ("Bypassed Defenders",   [2],                                           False),
        ("Deep Passes",          ["Deep Passes"],                               False),
        ("Advanced Touches",     ["Advanced Touches"],                          False),
        ("Final Third Bypasses", ["Final Third Bypasses"],                      False),
        ("Ball Losses",          [22],                                          True),
        ("Critical Ball Losses", [49],                                          True),
    ],

    # ── MIDFIELDER (11 metrics) ───────────────────────────────────────
    "Midfielder": [
        ("Bypassed Opponents",   [0],                                           False),
        ("Bypassed Defenders",   [2],                                           False),
        ("Pass Bypasses",        ["Pass Bypasses"],                             False),
        ("Pass Def Bypasses",    ["Pass Def Bypasses"],                         False),
        ("Bypassed Midfielders", [29],                                          False),
        ("Pass Accuracy %",      ["Pass Accuracy %"],                           False),
        ("Aerial Win %",         ["Aerial Win %"],                              False),
        ("Ground Duel Win %",    ["Ground Duel Win %"],                         False),
        ("Defensive Actions",    ["Defensive Actions"],                         False),
        ("Ball Losses",          [22],                                          True),
        ("Critical Ball Losses", [49],                                          True),
    ],

    # ── CENTERBACK (11 metrics) ───────────────────────────────────────
    "Centerback": [
        ("Aerial Win %",         ["Aerial Win %"],                              False),
        ("Ground Duel Win %",    ["Ground Duel Win %"],                         False),
        ("Defensive Actions",    ["Defensive Actions"],                         False),
        ("Bypassed Opponents",   [0],                                           False),
        ("Bypassed Defenders",   [2],                                           False),
        ("Pass Accuracy %",      ["Pass Accuracy %"],                           False),
        ("Pass Def Bypasses",    ["Pass Def Bypasses"],                         False),
        ("Clearance Bypasses",   [112],                                         False),
        ("Header Bypasses",      [113],                                         False),
        ("Ball Losses",          [22],                                          True),
        ("Critical Ball Losses", [49],                                          True),
    ],

    # ── FULLBACK (11 metrics) ─────────────────────────────────────────
    "Fullback": [
        ("Crosses",              ["Crosses"],                                   False),
        ("Cross Bypasses",       ["Cross Bypasses"],                            False),
        ("Bypassed Opponents",   [0],                                           False),
        ("Bypassed Defenders",   [2],                                           False),
        ("Aerial Win %",         ["Aerial Win %"],                              False),
        ("Ground Duel Win %",    ["Ground Duel Win %"],                         False),
        ("Defensive Actions",    ["Defensive Actions"],                         False),
        ("Pass Accuracy %",      ["Pass Accuracy %"],                           False),
        ("Pass Def Bypasses",    ["Pass Def Bypasses"],                         False),
        ("Final Third Bypasses", ["Final Third Bypasses"],                      False),
        ("Ball Losses",          [22],                                          True),
    ],

    # ── GOALKEEPER (10 metrics) ───────────────────────────────────────
    "Goalkeeper": [
        ("Pass Accuracy %",      ["Pass Accuracy %"],                           False),
        ("Bypassed Opponents",   [0],                                           False),
        ("Bypassed Defenders",   [2],                                           False),
        ("Pass Bypasses",        ["Pass Bypasses"],                             False),
        ("Pass Def Bypasses",    ["Pass Def Bypasses"],                         False),
        ("Offensive Touches",    [92],                                          False),
        ("Clearance Bypasses",   [112],                                         False),
        ("Ball Losses",          [22],                                          True),
        ("Critical Ball Losses", [49],                                          True),
        ("Own Goals",            [38],                                          True),
    ],
}

DEFAULT_PROFILES = ["Winger", "Striker", "Attacking Mid", "Midfielder", "Fullback", "Centerback", "Goalkeeper"]

POSITION_TO_PROFILES = {
    "CENTER_FORWARD":              ["Striker"],
    "LEFT_WINGER":                 ["Winger"],
    "RIGHT_WINGER":                ["Winger"],
    "LEFT_WINGBACK_DEFENDER":      ["Fullback"],
    "RIGHT_WINGBACK_DEFENDER":     ["Fullback"],
    "CENTER_BACK":                 ["Centerback"],
    "CENTRE_BACK":                 ["Centerback"],
    "CENTRAL_DEFENDER":            ["Centerback"],
    "GOALKEEPER":                  ["Goalkeeper"],
    "ATTACKING_MIDFIELD":          ["Attacking Mid"],
    "CENTRAL_MIDFIELD":            ["Midfielder"],
    "DEFENSE_MIDFIELD":            ["Midfielder"],
    "MIDFIELD":                    ["Midfielder"],
}

# Invert the map: profile_name → [pos_tokens…]
# Built once at module load so filter_for_profile can do a simple dict lookup.
PROFILE_TO_POSITIONS = {}
for _pos_key, _prof_list in POSITION_TO_PROFILES.items():
    for _p in _prof_list:
        PROFILE_TO_POSITIONS.setdefault(_p, []).append(_pos_key)

def build_profiles_from_csv(df: pd.DataFrame):
    profiles = {}
    cols = list(df.columns)
    for prof_name, metrics in PROFILE_TEMPLATES.items():
        built = []
        for label, candidates, invert in metrics:
            real = resolve_any(candidates, cols)
            if real is None:
                continue
            built.append((label, real, invert))
        if len(built) >= MIN_METRICS_PER_RADAR:
            profiles[prof_name] = built[:MAX_METRICS_PER_RADAR]
    return profiles

# ─────────────────────────────────────────────
# Radar math
# ─────────────────────────────────────────────
def percentile_rank(series, value, invert):
    s = pd.to_numeric(series, errors="coerce").dropna()
    if s.empty or pd.isna(value):
        return np.nan
    rank = (s < value).sum() + 0.5 * (s == value).sum()
    pct  = rank / len(s) * 100
    return 100 - pct if invert else pct

def pct_color(pct):
    if pd.isna(pct):  return TEXT_DIM
    if pct >= 70:     return PCT_HIGH
    if pct >= 30:     return PCT_MID
    return PCT_LOW

def fmt(v):
    if pd.isna(v): return "—"
    v = float(v)
    if abs(v) >= 100: return f"{v:.0f}"
    if abs(v) >= 10:  return f"{v:.1f}"
    if abs(v) >= 1:   return f"{v:.2f}"
    return f"{v:.3f}"

def axis_range(series):
    s = pd.to_numeric(series, errors="coerce").dropna()
    if s.empty: return (0, 1)
    lo, hi = float(s.quantile(0.05)), float(s.quantile(0.95))
    if hi == lo:
        lo, hi = float(s.min()), float(s.max())
    if hi == lo:
        hi = lo + 1
    return lo, hi

def norm01(value, lo, hi, invert=False):
    if pd.isna(value) or hi == lo: return np.nan
    x = np.clip((value - lo) / (hi - lo), 0, 1)
    return 1 - x if invert else float(x)

# ─────────────────────────────────────────────
# Drawing
# ─────────────────────────────────────────────
def draw_radar_polygons(ax, player_norm, league_norm, n):
    angles = np.linspace(0, 2 * np.pi, n, endpoint=False)
    ac = np.append(angles, angles[0])
    vp = np.append(player_norm, player_norm[0])
    vl = np.append(league_norm, league_norm[0])

    ax.fill(ac, vl, color=LEAGUE_COL, alpha=0.08, zorder=2)
    ax.plot(ac, vl, color=LEAGUE_COL, linewidth=1.8, linestyle="--", zorder=3)
    ax.fill(ac, vp, color=PLAYER_COL, alpha=0.18, zorder=4)
    ax.plot(ac, vp, color=PLAYER_COL, linewidth=2.5, zorder=5)
    ax.scatter(angles, player_norm, color=PLAYER_COL, s=30, zorder=6,
               edgecolors=BG, linewidths=1.2)

def draw_table(ax, labels, raw_vals, pcts, invert_flags):
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis("off")

    n        = len(labels)
    pad_top  = 0.065
    pad_bot  = 0.02
    row_h    = (1 - pad_top - pad_bot) / n

    ax.text(0.26, 1 - pad_top / 2, "METRIC", fontsize=7.5, color=TEXT_DIM,
            ha="center", va="center", fontweight="bold")
    ax.text(0.68, 1 - pad_top / 2, "VALUE",  fontsize=7.5, color=TEXT_DIM,
            ha="center", va="center", fontweight="bold")
    ax.text(0.88, 1 - pad_top / 2, "PCT",    fontsize=7.5, color=TEXT_DIM,
            ha="center", va="center", fontweight="bold")

    ax.plot([0.01, 0.99], [1 - pad_top, 1 - pad_top],
            color=ACCENT_LINE, linewidth=0.9, transform=ax.transAxes)

    for i in range(n):
        yc = 1 - pad_top - (i + 0.5) * row_h

        if i % 2 == 1:
            ax.add_patch(plt.Rectangle(
                (0.01, yc - row_h / 2), 0.98, row_h,
                facecolor=TABLE_ROW_ALT, edgecolor="none", transform=ax.transAxes))

        ax.plot([0.01, 0.99], [yc - row_h / 2, yc - row_h / 2],
                color=TABLE_BORDER, linewidth=0.25, transform=ax.transAxes)

        inv_tag = " ▼" if invert_flags[i] else ""
        ax.text(0.035, yc, labels[i] + inv_tag,
                fontsize=7.9, color=TEXT_LIGHT, ha="left", va="center")
        ax.text(0.68,  yc, fmt(raw_vals[i]),
                fontsize=8.0, color=TEXT_LIGHT, ha="center", va="center", fontweight="bold")

        pct     = pcts[i]
        pct_str = f"{int(round(pct))}" if not pd.isna(pct) else "—"
        ax.text(0.88, yc, pct_str,
                fontsize=8.2, color=pct_color(pct), ha="center", va="center", fontweight="bold")

# ─────────────────────────────────────────────
# Profile selection
# ─────────────────────────────────────────────
def profiles_for_player(pos_str: str, available_profiles):
    if TARGET_PROFILE:
        return [TARGET_PROFILE] if TARGET_PROFILE in available_profiles else []

    if not pos_str:
        return [p for p in DEFAULT_PROFILES if p in available_profiles][:3]

    tokens = [t.strip().upper() for t in pos_str.replace(";", ",").split(",") if t.strip()]
    chosen = []
    for t in tokens:
        for key, plist in POSITION_TO_PROFILES.items():
            if key in t:
                for p in plist:
                    if p in available_profiles and p not in chosen:
                        chosen.append(p)
    if not chosen:
        chosen = [p for p in DEFAULT_PROFILES if p in available_profiles]
    return chosen[:3]

# ─────────────────────────────────────────────
# Generate one radar PNG
# ─────────────────────────────────────────────
def generate_radar(df_comp_num, row, player_name, profile_name, profile_metrics, out_dir, pos_display):
    labels       = [m[0] for m in profile_metrics]
    col_names    = [m[1] for m in profile_metrics]
    invert_flags = [m[2] for m in profile_metrics]
    n = len(labels)

    player_vals = np.array([pd.to_numeric(row.get(c, np.nan), errors="coerce") for c in col_names], dtype=float)
    league_vals = np.array([pd.to_numeric(df_comp_num[c], errors="coerce").mean() for c in col_names], dtype=float)

    pcts = np.array([
        percentile_rank(df_comp_num[c], player_vals[i], invert_flags[i])
        for i, c in enumerate(col_names)
    ], dtype=float)

    ranges      = [axis_range(df_comp_num[c]) for c in col_names]
    player_norm = np.array([norm01(player_vals[i], *ranges[i], invert_flags[i]) for i in range(n)])
    league_norm = np.array([norm01(league_vals[i], *ranges[i], invert_flags[i]) for i in range(n)])
    player_norm = np.nan_to_num(player_norm, nan=0.0)
    league_norm = np.nan_to_num(league_norm, nan=0.0)

    fig = plt.figure(figsize=(16, 9), facecolor=BG)

    # top accent bar
    bar = fig.add_axes([0, 0.964, 1, 0.009])
    bar.set_facecolor(ACCENT_LINE)
    bar.set_xticks([]); bar.set_yticks([])
    for sp in bar.spines.values():
        sp.set_visible(False)

    # header
    squad = str(row.get("squadName", "")) if "squadName" in row else ""
    fig.text(0.035, 0.935, player_name, fontsize=24, color=PLAYER_COL,
             fontweight="bold", va="top")
    subtitle = "  •  ".join([s for s in [squad, f"Pos: {pos_display}"] if s])
    fig.text(0.035, 0.900, subtitle or "Iteration 1421", fontsize=12, color=TEXT_DIM, va="top")

    fig.text(0.965, 0.935, profile_name.upper() + " PROFILE", fontsize=15,
             color=TEXT_LIGHT, va="top", ha="right", fontweight="bold")
    fig.text(0.965, 0.905, "IMPECT Iteration 1421", fontsize=9, color=TEXT_DIM,
             va="top", ha="right")

    # radar
    ax_r = fig.add_axes([0.01, 0.10, 0.56, 0.64], polar=True)
    ax_r.set_facecolor(RADAR_BG)

    angles = np.linspace(0, 2 * np.pi, n, endpoint=False)
    ax_r.set_theta_offset(np.pi / 2)
    ax_r.set_theta_direction(-1)
    ax_r.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    ax_r.set_yticklabels([""] * 5)
    ax_r.set_ylim(0, 1.0)
    ax_r.yaxis.grid(True, color=GRID_COLOR, linewidth=0.5)
    ax_r.xaxis.grid(True, color=GRID_COLOR, linewidth=0.5)
    ax_r.spines["polar"].set_visible(False)
    ax_r.set_xticks(angles)
    ax_r.set_xticklabels(labels, fontsize=7.6, color=TEXT_LIGHT)
    ax_r.tick_params(pad=9)

    draw_radar_polygons(ax_r, player_norm, league_norm, n)

    # stats table
    ax_t = fig.add_axes([0.595, 0.10, 0.38, 0.64])
    ax_t.set_facecolor(BG)
    draw_table(ax_t, labels, player_vals, pcts, invert_flags)

    # footer
    fig.text(0.035, 0.038, "IMPECT", fontsize=20, color=ACCENT_LINE,
             fontweight="bold", alpha=0.35, va="bottom")
    fig.text(0.035, 0.012,
             "CSV-only  •  per match  •  ▼ = lower is better",
             fontsize=6.5, color=TEXT_DIM, va="bottom")

    os.makedirs(out_dir, exist_ok=True)
    safe     = lambda s: "".join(c if c.isalnum() or c in " _-" else "" for c in s).replace(" ", "_")
    out_path = os.path.join(out_dir, f"{safe(player_name)}_{safe(profile_name)}.png")
    fig.savefig(out_path, dpi=200, bbox_inches="tight", facecolor=BG)
    plt.close(fig)
    return out_path

# ─────────────────────────────────────────────
# MAIN
# ─────────────────────────────────────────────
def main():
    print("Loading CSV …")
    df = load_csv(CSV_PATH)
    print(f"  {df.shape[0]} players × {df.shape[1]} columns")

    print("Computing derived metrics …")
    df = compute_derived(df)

    pos_col = positions_column(df)
    print(f"Positions column: {pos_col or 'NOT FOUND'}")

    profiles = build_profiles_from_csv(df)
    if not profiles:
        raise RuntimeError("No profiles could be built from your CSV (too many KPIs missing).")

    print("Profiles built from CSV:")
    for k, v in profiles.items():
        print(f"  - {k}: {len(v)} metrics")

    # Collect every column needed (KPIs + metadata)
    needed_cols = set()
    for prof_metrics in profiles.values():
        for _, c, _ in prof_metrics:
            needed_cols.add(c)
    if pos_col:
        needed_cols.add(pos_col)
    needed_cols.update(["commonname", "firstname", "lastname", "squadName"])

    # prep_numeric skips META_COLS internally
    df_num = prep_numeric(df, needed_cols)

    # ── player filter ──
    if TARGET_PLAYER:
        rows = find_player_rows(df_num.copy(), TARGET_PLAYER)
        if rows.empty:
            token  = _norm_name(TARGET_PLAYER).split(" ")[-1]
            approx = df_num[
                df_num.get("commonname", pd.Series(dtype=str)).map(_norm_name).str.contains(token, na=False)
                | df_num.get("lastname",   pd.Series(dtype=str)).map(_norm_name).str.contains(token, na=False)
            ][["commonname", "firstname", "lastname", "positions", "squadName"]].head(20)
            print("\n⚠️  TARGET_PLAYER not found. Closest matches:")
            print(approx.to_string(index=False))
            raise RuntimeError(f"TARGET_PLAYER not found: {TARGET_PLAYER!r}")
    else:
        rows = df_num

    if rows.empty:
        raise RuntimeError("No rows to process after filtering.")

    made = 0
    for _, row in rows.iterrows():
        player_name = str(row.get("commonname", "")).strip()
        if not player_name:
            player_name = (str(row.get("firstname", "")).strip() + " " +
                           str(row.get("lastname",  "")).strip()).strip()
        if not player_name:
            continue

        pos_str     = normalize_positions_str(row.get(pos_col, "")) if pos_col else ""
        pos_display = pos_str if pos_str else "—"

        prof_list = profiles_for_player(pos_str, set(profiles.keys()))

        if TARGET_PROFILE and TARGET_PROFILE not in profiles:
            raise RuntimeError(f"TARGET_PROFILE {TARGET_PROFILE!r} not built from CSV columns.")

        for prof_name in prof_list:
            prof_metrics = profiles.get(prof_name, [])
            if len(prof_metrics) < MIN_METRICS_PER_RADAR:
                continue
            # ── filter comp pool to players who carry a position that maps
            #    to THIS profile, not the target player's primary position ──
            df_comp = filter_for_profile(df_num, pos_col, prof_name) if BENCHMARK_BY_POSITION else df_num
            out = generate_radar(df_comp, row, player_name, prof_name, prof_metrics, OUT_DIR, pos_display)
            made += 1
            print("✅", out)

    print(f"\n✅ Done. Generated {made} PNG(s) in: {OUT_DIR}")

if __name__ == "__main__":
    main()

Loading CSV …
  445 players × 1430 columns
Computing derived metrics …
Positions column: positions
Profiles built from CSV:
  - Striker: 12 metrics
  - Winger: 12 metrics
  - Attacking Mid: 12 metrics
  - Midfielder: 11 metrics
  - Centerback: 11 metrics
  - Fullback: 11 metrics
  - Goalkeeper: 10 metrics
✅ /Users/user/IMPECT/impect_outputs/Lukas_Letenay_Attacking_Mid.png
✅ /Users/user/IMPECT/impect_outputs/Lukas_Letenay_Midfielder.png
✅ /Users/user/IMPECT/impect_outputs/Lukas_Letenay_Striker.png

✅ Done. Generated 3 PNG(s) in: /Users/user/IMPECT/impect_outputs
